<img src="docs/sa_logo.png" width="250" align="left">

#  Name

This tutorial walks you through the ```task name e.g Named Entity Recognition``` automation pipeline within SuperAnnotate platform. 

Provide rough descriptions for each step of the tutorial

1. Setup the environment and install dependences
2. Run the Hugging Face model ```model name e.g something QANer``` model on unlabeled data
3. Setup ```Text``` project at SuperAnnotate with corresponding classes and data
4. Upload model predictions to the platform 


## 1.  Environment Setup

In [ ]:
#sa installation
! pip install superannotate 

SuperAnnotate Python SDK functions work within the team scope of the platform, so a team-level authorization is required. To authorize the package in a given team scope, get the authorization token from the team settings page.

In [ ]:
#sa authorization
from superannotate import SAClient
SDK_TOKEN = "ADD_YOUR_TOKEN_HERE"

sa = SAClient(token = SDK_TOKEN)

## 2. Model Prediction

In [ ]:
import easyocr as eo
import json

# get esayocr english model
reader = eo.Reader(['en'])

#iterate over items, generate annotation.json from predictions
for item in data:
    img = cv2.imread(item)
    results = reader.readtext(img)
    # Get all words from the prediction that have more than 0.1 confidence
    words = [prediction[1] for prediction in results if prediction[2] > 0.1]
    # Make a line from the predicted words
    line = ' '.join(words) + '\n'
    # Set the line as pointLabel of the bounding box
    annotion_json = {"metadata": {"name": name}, "instances": {"type": "tag", "attributes": [line]}}
    
    #write to disk
    with open(f'./annotations/{name}___objects.json', 'w') as outfile:
        json.dump(annotation_json, outfile)
    

## 2*. Model Training

In [ ]:
eo.train(dataset, ner_classes)

## 3. Project Setup

In [ ]:
# 1. create text project
project = "Document Processing"
sa.create_project(project, "---", "Text")

#2. create annotation classes
sa.create_annotation_classes(project, classes)

#3. attach dataset
sa.attach_items(project, "./data.csv")

## 4. Import Predictions

In [ ]:
#1. Upload constructed annotation JSONs back to the platform.
sa.upload_annotations_from_folder_to_project(project, "./annotations")